In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm.notebook import trange
import firedrake
from firedrake import inner, Constant, div, dx, ds, exp
import icepack, icepack.models, icepack.solvers
from icepack.constants import (
    ice_density as ρ_I,
    water_density as ρ_W,
    weertman_sliding_law as m,
    gravity as g,
)

In [ ]:
Lx = Constant(50e3)
nx = 50
interval = firedrake.UnitIntervalMesh(nx)
ξ, = firedrake.SpatialCoordinate(interval)
V = interval.coordinates.function_space()
λ = Constant(0.125)
expr = firedrake.as_vector((Lx * (λ * ξ + (1 - λ) * ξ**2),))
x = firedrake.Function(V).interpolate(expr)
mesh = firedrake.Mesh(x)

Q = firedrake.FunctionSpace(mesh, "CG", 2)
V = firedrake.FunctionSpace(mesh, "CG", 2)

x, = firedrake.SpatialCoordinate(mesh)

In [ ]:
fig, ax = plt.subplots()
xs = mesh.coordinates.dat.data_ro
ax.scatter(np.linspace(0, len(xs), len(xs)), xs)

In [ ]:
b_in, b_out = Constant(200), Constant(-400)
δb = Constant(150)
λ = Constant(0.85)
α = Constant(100.0)
expr = b_in - (b_in - b_out) * x / Lx + δb * exp(-α * (x / Lx - λ)**2)
b = firedrake.Function(Q).interpolate(expr)

s_in, s_out = Constant(850), Constant(50)
s0 = firedrake.Function(Q).interpolate(s_in - (s_in - s_out) * x / Lx)

h0 = firedrake.Function(Q).interpolate(s0 - b)

In [ ]:
h_in = s_in - b_in
δs_δx = (s_out - s_in) / Lx
τ_D = -Constant(ρ_I * g * h_in * δs_δx)

u_in, u_out = Constant(20), Constant(2400)
velocity_x = u_in + (u_out - u_in) * (x/Lx)**2
u0 = firedrake.Function(V).interpolate(velocity_x)

In [ ]:
T = Constant(255.0)
A = icepack.rate_factor(T)

C_0, δC = Constant(0.95), Constant(0.05)
expr = (C_0 - δC * x/Lx) * τ_D / u_in ** (1 / m)
C = firedrake.Function(Q).interpolate(expr)

p_W = ρ_W * g * firedrake.max_value(0, h0 - s0)
p_I = ρ_I * g * h0
ϕ = 1 - p_W / p_I

In [ ]:
def weertman_friction_with_ramp(**kwargs):
    names = ("velocity", "thickness", "surface", "friction")
    u, h, s, C = map(kwargs.__getitem__, names)
    p_W = ρ_W * g * firedrake.max_value(0, h - s)
    p_I = ρ_I * g * h
    ϕ = 1 - p_W / p_I
    return icepack.models.friction.bed_friction(velocity=u, friction=C * ϕ)

model = icepack.models.IceStream(friction=weertman_friction_with_ramp)
opts = {"dirichlet_ids": [1]}
solver = icepack.solvers.FlowSolver(model, **opts)

In [ ]:
u0 = solver.diagnostic_solve(
    velocity=u0, thickness=h0, surface=s0, fluidity=A, friction=C
)

In [ ]:
num_years = 250
timesteps_per_year = 2

δt = 1.0 / timesteps_per_year
num_timesteps = num_years * timesteps_per_year

a_0, δa = Constant(1.2), Constant(2.7)
a = firedrake.Function(Q).interpolate(a_0 - δa * x / Lx)

h = h0.copy(deepcopy=True)
s = s0.copy(deepcopy=True)
u = u0.copy(deepcopy=True)

hs = [h.copy(deepcopy=True)]
ss = [s.copy(deepcopy=True)]
zbs = [firedrake.Function(Q).interpolate(s - h)]

for step in trange(num_timesteps + 1):
    h = solver.prognostic_solve(
        δt,
        thickness=h,
        accumulation=a,
        velocity=u,
        thickness_inflow=h0,
    )
    s = icepack.compute_surface(thickness=h, bed=b)

    u = solver.diagnostic_solve(
        velocity=u,
        thickness=h,
        surface=s,
        fluidity=A,
        friction=C,
    )

    hs.append(h.copy(deepcopy=True))
    ss.append(s.copy(deepcopy=True))
    zbs.append(firedrake.Function(Q).interpolate(s - h))

In [ ]:
%%capture
fig, ax = plt.subplots()

def animate(fields):
    h, s, z_b = fields
    ax.clear()
    firedrake.plot(z_b, axes=ax, edgecolor="tab:blue")
    firedrake.plot(b, axes=ax, edgecolor="tab:brown")
    firedrake.plot(s, axes=ax, edgecolor="tab:blue")

animation = FuncAnimation(fig, animate, list(zip(hs, ss, zbs)), interval=1e3 / 60)

In [ ]:
HTML(animation.to_html5_video())

In [ ]:
hs = [h.copy(deepcopy=True)]
ss = [s.copy(deepcopy=True)]
zbs = [firedrake.Function(Q).interpolate(s - h)]

h_min = Constant(10.0)
a_0.assign(1.05)
a.interpolate(a_0 - δa * x / Lx)
for step in trange(4 * num_timesteps + 1):
    h = solver.prognostic_solve(
        δt, thickness=h, accumulation=a, velocity=u, thickness_inflow=h0
    )
    h.interpolate(firedrake.max_value(h_min, h))
    s = icepack.compute_surface(h=h, b=b)
    u = solver.diagnostic_solve(
        velocity=u, thickness=h, surface=s, fluidity=A, friction=C
    )

    hs.append(h.copy(deepcopy=True))
    ss.append(s.copy(deepcopy=True))
    zbs.append(firedrake.Function(Q).interpolate(s - h))

In [ ]:
%%capture
fig, ax = plt.subplots()

def animate(fields):
    h, s, z_b = fields
    ax.clear()
    firedrake.plot(z_b, axes=ax, edgecolor="tab:blue")
    firedrake.plot(b, axes=ax, edgecolor="tab:brown")
    firedrake.plot(s, axes=ax, edgecolor="tab:blue")

animation = FuncAnimation(fig, animate, list(zip(hs, ss, zbs)), interval=1e3 / 60)

In [ ]:
HTML(animation.to_html5_video())